**Descripción del conjunto de datos**

El conjunto de datos para esta competencia (tanto de entrenamiento como de prueba) se generó a partir de un modelo de aprendizaje profundo entrenado en Credit Card Fraud Detection . Las distribuciones de características son similares, pero no exactamente iguales, a las del original. Siéntase libre de usar el conjunto de datos original como parte de esta competencia, tanto para explorar las diferencias como para ver si incorporar el original en el entrenamiento mejora el rendimiento del modelo.

Tenga en cuenta que este conjunto de datos base para esta competencia era mucho más grande que los conjuntos de datos anteriores de Tabular Tuesday y, por lo tanto, puede contener más artefactos que las últimas tres competencias.

**Archivos:**

    train.csv - el conjunto de datos de entrenamiento; Classes el objetivo
    test.csv - el conjunto de datos de prueba; tu objetivo es predecirClass
    sample_submission.csv : un archivo de envío de muestra en el formato correcto


**Documentación:**

El conjunto de datos contiene transacciones realizadas con tarjetas de crédito en septiembre de 2013 por titulares de tarjetas europeos.
Este conjunto de datos presenta transacciones que ocurrieron en dos días, donde tenemos 492 fraudes de 284,807 transacciones. El conjunto de datos está muy desequilibrado, la clase positiva (fraudes) representa el 0,172 % de todas las transacciones.

Contiene solo variables de entrada numéricas que son el resultado de una transformación PCA. Desafortunadamente, debido a problemas de confidencialidad, no podemos proporcionar las funciones originales ni más información general sobre los datos. Las características V1, V2, … V28 son los principales componentes obtenidos con PCA, las únicas características que no han sido transformadas con PCA son 'Tiempo' y 'Cantidad'. La característica 'Tiempo' contiene los segundos transcurridos entre cada transacción y la primera transacción en el conjunto de datos. La función 'Cantidad' es la cantidad de la transacción, esta función se puede utilizar para el aprendizaje sensible a los costos dependiente del ejemplo. Feature 'Class' es la variable de respuesta y toma valor 1 en caso de fraude y 0 en caso contrario.

Dada la relación de desequilibrio de clase, recomendamos medir la precisión utilizando el área bajo la curva de recuperación de precisión (AUPRC). La precisión de la matriz de confusión no es significativa para la clasificación desequilibrada.


**Evaluación:**

Las presentaciones se evalúan en el área bajo la curva ROC entre la probabilidad pronosticada y el objetivo observado.

Para cada uno iden el conjunto de prueba, debe predecir el valor del objetivo Class. El archivo debe contener un encabezado y tener el siguiente formato:

    id,Class
    341588,0.23
    341589,0.92
    341590,0.02
    etc.

**REFERENCIAS:**

Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson y Gianluca Bontempi. Calibración de probabilidad con submuestreo para clasificación desequilibrada. En Simposio sobre Inteligencia Computacional y Minería de Datos (CIDM), IEEE, 2015

Dal Pozolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Lecciones aprendidas en la detección de fraudes con tarjetas de crédito desde la perspectiva de un profesional , Sistemas expertos con aplicaciones, 41,10,4915-4928,2014, Pergamon

Dal Pozolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, César; Bontempi, Gianluca. Detección de fraude con tarjetas de crédito: un modelo realista y una estrategia de aprendizaje novedosa, transacciones IEEE en redes neuronales y sistemas de aprendizaje, 29,8,3784-3797,2018, IEEE

Dal Pozzolo, Andrea Adaptive Machine learning para la detección de fraudes con tarjetas de crédito Tesis doctoral de ULB MLG (supervisada por G. Bontempi)

Carcillo, Fabricio; Dal Pozolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: un marco escalable para transmitir la detección de fraudes con tarjetas de crédito con Spark , Information fusion,41, 182-194,2018,Elsevier

Carcillo, Fabricio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Transmisión de estrategias de aprendizaje activo para la detección de fraudes con tarjetas de crédito en la vida real: evaluación y visualización, International Journal of Data Science and Analytics, 5,4,285-300,2018, Springer International Publishing

Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Técnicas de adaptación de dominios de aprendizaje profundo para la detección de fraudes con tarjetas de crédito , INNSBDDL 2019: Avances recientes en Big Data y aprendizaje profundo, págs. 78-88, 2019

Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combinación de aprendizaje no supervisado y supervisado en ciencias de la información de detección de fraude con tarjetas de crédito , 2019

Yann-Aël Le Borgne, Gianluca Bontempi Aprendizaje automático reproducible para la detección de fraudes con tarjetas de crédito - Manual práctico

Bertrand Lebichot, Gianmarco Paldino, Wissam Siblini, Liyun He, Frederic Oblé, Gianluca Bontempi Estrategias de aprendizaje incremental para la detección de fraudes con tarjetas de crédito , International Journal of Data Science and Analytics


# Librerias

In [6]:
import pandas as pd
from sklearn.metrics import average_precision_score
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)

# Lectura dataset

In [10]:
df_train = pd.read_csv(r'/Users/aitorelordizamora/Library/Mobile Documents/com~apple~CloudDocs/Datasets/datasets fraude/train.csv', sep=',')
df_test = pd.read_csv(r'/Users/aitorelordizamora/Library/Mobile Documents/com~apple~CloudDocs/Datasets/datasets fraude/test.csv', sep=',')

In [11]:
df_test

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,219129,120580.0,2.115519,-0.691809,-1.305514,-0.685655,-0.641265,-0.764784,-0.924262,-0.023030,-0.230126,0.220610,1.058325,-0.723363,-0.619241,-2.243711,-0.271429,2.007845,1.213534,0.282344,0.579121,0.067367,0.241708,0.682524,0.037769,-0.546859,-0.123055,-0.084889,0.004720,-0.021944,29.95
1,219130,120580.0,1.743525,-1.681429,-0.547387,-1.061113,-0.695825,2.458824,-1.632859,1.073529,1.068183,0.483337,0.274519,-0.346514,-0.409519,-0.564673,-0.143855,1.334855,0.160901,-0.087356,-0.066264,0.441788,0.543278,1.294571,0.309541,3.703925,-0.242579,0.068708,0.002629,0.064690,163.50
2,219131,120580.0,2.205568,-1.571445,-0.238965,-1.662517,-1.652324,-0.054701,-1.682064,0.105613,-1.177858,1.626352,0.601148,-0.041610,0.432748,-0.527684,-0.543665,0.043896,0.058472,0.622667,0.010671,-0.366906,-0.131527,0.086623,0.291375,0.739087,-0.543006,-0.297813,0.043699,-0.037855,16.00
3,219132,120580.0,1.989728,-0.972909,-1.938259,-1.440129,-0.166855,-0.794048,0.252889,-0.399789,2.079398,-1.225592,-0.869368,1.647638,1.138026,-0.258468,-0.361765,-1.253326,0.099479,-0.587702,1.207085,-0.049136,-0.080115,-0.010732,-0.038550,0.656830,0.343470,-0.627529,-0.024338,-0.036143,120.98
4,219133,120580.0,-1.943548,-1.668761,0.363601,-0.977610,2.684779,-2.037681,0.039709,-0.048895,-0.281749,-0.341879,0.471628,0.637604,-0.592416,0.331702,-1.451799,0.096467,-0.853439,-0.462060,-0.267098,0.391627,0.083389,-0.306918,0.247822,-0.391799,-0.790716,-0.025706,0.330758,0.335537,1.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146082,365211,172789.0,-0.661834,-0.622980,0.735580,-0.554067,0.869889,-0.389502,0.161226,0.069945,0.698829,-0.658935,-2.129855,-0.655069,-0.551589,-0.143536,-0.329767,0.625951,-0.949933,0.164734,0.079083,0.262408,-0.046068,-0.425692,0.253833,-1.076331,-0.999877,0.161938,0.149243,0.279708,94.81
146083,365212,172789.0,-0.098889,0.761900,0.340012,-0.589546,0.717091,-0.262765,0.726272,-0.070608,0.095958,-0.776078,-1.130049,0.506635,1.239831,-0.039782,0.446481,-0.049086,-0.746027,0.361708,-0.191256,0.064640,0.441215,1.284819,-0.141790,0.646930,-0.419964,-0.561426,0.326700,0.255166,24.99
146084,365213,172789.0,-1.487918,1.769142,0.551671,0.361566,0.056347,0.084844,0.567545,-0.059899,0.331706,1.623577,0.807877,1.021134,1.069455,-0.322923,0.063989,-0.089844,-0.764603,0.739254,0.291846,0.792736,0.163500,1.287079,-0.186955,-0.349847,-0.161724,-0.213933,0.392745,0.070938,29.99
146085,365214,172790.0,-1.889979,-0.154869,0.068496,-1.906964,-1.743181,0.717490,0.981608,-0.209713,-0.556087,-0.283025,-1.664145,-0.559516,1.238936,-0.887569,-1.155718,1.379725,0.114537,-1.190413,0.664771,-0.513829,0.121457,0.764642,-0.217844,0.104599,-0.008906,-0.204957,-0.752326,-0.155728,314.00


In [4]:
# Control valores nulos

value_null = df_train.isnull().any().any()
print("VALORES NULOS:",value_null)


# Control valores duplicados

value_dupli = df_train.duplicated().any()
print("VALORES DUPLICADOS:",value_dupli)

VALORES NULOS: False
VALORES DUPLICADOS: False


In [12]:

# Separar características y variable objetivo
X = df_train.drop(['id', 'Class'], axis=1)  # Características
y = df_train['Class']  # Variable objetivo

# Crear y entrenar el modelo
modelo_regresion = LinearRegression() 
modelo_regresion.fit(X, y)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo_regresion.predict(df_train.drop(['id', 'Class'], axis=1))

# Calcular el AUPRC
auprc_score = average_precision_score(df_train['Class'], y_pred)

print(f"AUPRC: {auprc_score:.4f}")

AUPRC: 0.0469


In [13]:
y_pred

array([-0.00026087,  0.00120489,  0.00457459, ...,  0.00745124,
        0.00226134,  0.00213234])